In [ ]:
#DATA 
# to import data from google map - export map as KML then convert to geojson using an online converter
# each category of points will be a file

In [ ]:
# %pip install pygris
# Spatial Analysis
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import folium
import pygris as pygris
import folium.plugins as fpl

In [ ]:
# Vernon County Mapping

In [ ]:
# map the county as a base

In [ ]:
# add parks and reserves 
parks = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Parks_and_Reserves.geojson")
# add libraries
libraries = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Libraries.geojson")
#add medical centers
medical_centers = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Hospitals_and_Clinics.geojson")
# add high schools
city_HS = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Vernon_Cty_High_Schools.geojson")
# add villages and cities w/o charges
no_charger = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Villages_and_Cities_without_charging.geojson")
# mark L2 chargers
l2_charger = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Existing_L2_Chargers.geojson")
# mark DC fast chargers
dc_charger = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Existing_DC_Fast_Chargers.geojson")
# mark potential sites
potential_dc_charger = gpd.read_file("Vernon_County_EV_Charger_Recommendations/Potential_DC_Fast_Chargers.geojson")

In [ ]:
# wi_counties = pygris.counties(state="WI", cb=True, cache=True)
# wi_counties[wi_counties["NAME"] == "Vernon"].plot()

In [ ]:
#stitch the dfs together
parks['color'] = "brown"
parks['icon'] = "tent"
libraries['color'] = "blue"
libraries['icon'] = "book"
medical_centers['icon'] = "hospital"
medical_centers['color'] = "red"
city_HS["color"] = "green"
city_HS["icon"] = "school"
l2_charger["color"] = "yellow"
l2_charger["icon"] = "plug"
dc_charger["color"] = "orange"
dc_charger["icon"] = "bolt"
no_charger["color"] = "gray"
no_charger["icon"] = "x"
potential_dc_charger["color"] = "lightblue"
potential_dc_charger["icon"] = "question"

provided_POIs = pd.concat([parks, libraries, medical_centers, city_HS, l2_charger, dc_charger, no_charger, potential_dc_charger], axis=0)

In [ ]:
wi_counties = pygris.counties(state="WI", cache=True)
vernon_outline = wi_counties[wi_counties['NAME'] == "Vernon"]
vernon_outline = vernon_outline.to_crs(epsg=4326)
vernon_outline

In [ ]:
#https://geopandas.org/en/stable/gallery/plotting_with_folium.html
map_vernon = folium.Map([43.44476, -90.76852], zoom_start = 10)

# Create a geometry list from the GeoDataFrame
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in provided_POIs.geometry]

# Iterate through list and add a marker for each volcano, color-coded by its type.
i = 0
for coordinates in geo_df_list:
    # icon = fpl.BeautifyIcon(
    #     icon=provided_POIs.iloc[i]['icon'], border_color=provided_POIs.iloc[i]['color'], text_color=provided_POIs.iloc[i]['color'], icon_shape="square", inner_icon_style='margin-top:0;'
    # )
    # Place the markers with the popup labels and data
    folium.Marker(
            location=coordinates,
            popup=folium.Popup("Name: "
            + str(provided_POIs.iloc[i]['Name'])
            + "<br>"
            + "Description: "
            + str(provided_POIs.iloc[i]['description'])),
            # icon=icon
            icon = folium.Icon(color="%s" % provided_POIs.iloc[i]['color'])
        ).add_to(map_vernon)
    i = i + 1

#add the county outline
#https://geopandas.org/en/stable/gallery/polygon_plotting_with_folium.html
for _, r in vernon_outline.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "transparent"})
    # folium.Popup(r["BoroName"]).add_to(geo_j)
    geo_j.add_to(map_vernon)

map_vernon

In [ ]:
#work out the icons?
#add jobs, demographics, housing?